In [ ]:
from trello import TrelloListMonitor
from IFTTT import IFTTTNotifier
from habitica import HabiticaAPI

In [ ]:
monitor = TrelloListMonitor()
notifier = IFTTTNotifier()
habitica = HabiticaAPI()

In [ ]:
response = habitica.get_profile()
data = response['data']

data['stats']

exp = data['stats']['exp']
gp = data['stats']['gp']
level = data['stats']['lvl']

print(f"Level: {level}, Experience: {exp}, Gold: {gp}")

In [ ]:
def handle_trello_changes(diff):
    """Handle all the business logic when Trello changes"""
    if diff['added']:
        for card in diff['added']:            
            card_id = card['id']

            card_details = monitor.get_card_details(card_id)
            card_title = card_details['title']
            card_story_points = card_details.get('story_points')
            card_frontend_url = card_details.get('frontend_url')

            notifier.send_notification(title=f'Logging Trello Card #{card_title}#...',
                                       message=f'',
                                       url="", image_url="")


            # print(card_title, card_story_points, card_frontend_url)    

            habitica_info = habitica.log_story_points(card_story_points)
            exp, gp, level = habitica_info['stat_deltas']['exp'], \
                             habitica_info['stat_deltas']['gp'], \
                             habitica_info['stat_deltas']['level']
            
            # we want the title to be dynamic based on exp gp and level, although level might not change
            # emojis used for compression and concision
            # the emoji for exp is the star ⭐
            # the emoji for gold is the coin 🪙
            # the emoji for level is the up arrow ⬆️
            # and we only include the level, gp and exp if they have changed
            title_parts = []
            if exp:
                title_parts.append(f'⭐{round(exp, 2) if isinstance(exp, float) else exp}')
            if gp:
                title_parts.append(f'🪙{round(gp, 2) if isinstance(gp, float) else gp}')
            if level:
                title_parts.append(f'⬆️{level}')

            title = " | ".join(title_parts)

            notifier.send_notification(title='Trello Card Logged! - ' + title,
                                       message=f'specifically this card: ##{card_title}## with story points: {card_story_points}',
                                       url=card_frontend_url)



In [ ]:
monitor.monitor(interval=1,callback=handle_trello_changes)